# Stokes coefficients estimation
In this set of notebooks, we show simulations of retrival of a set of real spherical harmonics gravity coefficients from a set of gravitational measurements (either potential or acceleration). This is a simplified example of what is performed in an orbit determination (OD) campaign, where the Stokes coefficients are estimated, along with other parameters, from a set of radio-tracking measurements of an orbiting spacecraft.

### **Case 3**: Fitting gravity acceleration from points over an eccentric orbit
Here we further degrade the original, ideal scenario, by selecting an orbit with high eccentricity, which therefore provides uneven coverage.

In [1]:
import os
from time import time

import numpy as np
import plotly.graph_objects as go
import pyshtools as sh
import matplotlib.pyplot as plt
import pickle as pk
import json

import scripts
from scripts._units import *

from IPython.display import IFrame
import plotly.io as pio
pio.templates.default = "plotly_white"

### Settings

In [2]:
out_name = "eccentric_trajectory" # outputs destination folder (a subfolder of out)
out_path = os.path.join("out", out_name)
if not os.path.exists(out_path):
    os.makedirs(out_path)

use_potential = False # Whether to work with potential (flag to True) or with accelerations (flag to False)
perturb_msr = False # Whether to introduce additive Gaussian noise in the syntehtic measurements
drop_traj_points = True # Only keep locations of measurement points, to save on memory 
compute_covariance = True # Additionally compute uncertainties for the estimated coefficients (~20% increase in inversion time)
check_acc_msr = False # Compare acceleartion measurements with those given by pyshtools

msr_dim = 1 if use_potential else 3 # dimensions of each measurerment (i.e. scalar or vector)
partials_func = scripts.pot_cnm_partials if use_potential else scripts.acc_cnm_partials # function to use for partials computation
val_lbl = "U_grav (m^2/s^2)" if use_potential else "|a_grav| (m/s^2)" # label for plots
rng = np.random.default_rng()

### Ground-truth gravity
As in the previous notebook, we select as ground-truth the gravity coefficients of Mars, as estimated by Konopliv, et al. (2016), and available on the [NASA PDS](https://pds-geosciences.wustl.edu/mro/mro-m-rss-5-sdp-v1/mrors_1xxx/data/shadr/). 

In [3]:
# SH degree cutoff (for both simulation and estimation, and in any case limited by that of the file)
n_max_sim = 120 # macimum degree of the coefficients used to simulate the measurements
n_max_est = 100 # maximum degree of the coefficients to be estimated
n_max = max(n_max_sim, n_max_est)

# Read SH coefficients, and cut to n_max_sim
file_name = "spice/data/jgmro_120d_sha.tab"
cnm_mat, r_0, gm = scripts.read_SHADR(file_name)
cnm_mat_sim = cnm_mat[:, : n_max_sim + 1, : n_max_sim + 1]

# Index array and dictionary to go from a matrix of Cnm to a 1D array, and vice-versa
cnm_idx_sim, cnm_map_sim = scripts.get_cnm_idx(n_max_sim)
cnm_idx_est, cnm_map_est = scripts.get_cnm_idx(n_max_est)

### Measurements schedule
This section is used to pick measurement points among those describing a trajctory. The choice of the measurement time-tags is inspired by tracking schedules for radio-tracking observations. For example, we can select to pick 4 hours (`pass_length`) of contiguous points, 60-s apart from each other (`dt`), once every day (`passes_per_day=1`) and at random times during each day (`random_track_start=True`). All this over 1 Earth year (`t_span`).

In [4]:
t_vec = scripts.measurement_schedule(
        t_span=365 * day, dt = 60.* sec, passes_per_day=1, pass_length=4 * hour, random_track_start=True, rng=rng
    )
n_msr_pts = t_vec.shape[0]
print("{:d} measurements selected over {:.2f} days".format(n_msr_pts, t_vec[-1]/day))    

100%|█████████████████████████████████████| 365/365 [00:00<00:00, 537542.47it/s]

87600 measurements selected over 364.77 days


### Trajectory definition
The orbital elements input to this function are those at time 0. At all time-tags contained in the vector `t_vec`, the orbital elements are converted to cartesian and to spherical coordinates using SPICE. The only time-varying orbital elements are the mean anomaly (angular position along the orbit) and $\Omega$, which varies with a rate that allows the orbital plane to perform `coverage` rotations around the body in `t_vec[-1]` seconds.

In [5]:
e_mex = 0.57
r_p_mex = 298*km+r_0
a_mex = r_p_mex/(1-e_mex)
state_mat, sph_coords, t_vec = scripts.points_from_keplerian_traj(
    gm,
    t_vec,  # duration of the orbit
    coverage=1.0,  # no. of times orbital plane sweeps the whole body during t_span
    elts=dict(
        a=a_mex, i=92.6 * deg, e=e_mex, om=270 * deg, Om=0.0 * deg, M_0=0.0 * deg
    ), # keplerian elements (defaults to the ones of MRO)
)

## Forward model

### Simulating measurements
Given the measurements locations and the matrix of Stokes coefficients, the gravity measurements can be simulated following the equations in the previous notebook. If the simulation and estimation sets of coefficients have the same maximum degree, however, the computation of the synthetic measurements is performed in the next cell, at the same time as the computation of the partials.


In [6]:
sim_msr_vec = None
if n_max_sim != n_max_est:
    print("Computing synthetic measurements...")
    ts = time()

    # Compute potential and acceleration at the selected coordinates
    pot_sim, acc_sim = scripts.compute_pot_acc(
        cnm_mat_sim, r_0, sph_coords
    )
    pot_sim *= gm
    acc_sim *= gm
    # Pick right observable, and convert to 1D array
    if use_potential:
        sim_msr_vec = pot_sim.reshape(n_msr_pts * msr_dim)
    else:
        sim_msr_vec = acc_sim.reshape(n_msr_pts * msr_dim)

    # A little hacky, but for now this is stored as a bool to economize on memory
    cnm_mat_est = np.empty((2, n_max_est + 1, n_max_est + 1), dtype=bool)

    # Compare the obtained acceleration values with those computed by SHTOOLS
    if check_acc_msr:
        sh_coeffs = sh.SHGravCoeffs.from_array(cnm_mat_sim, gm, r_0)
        acc_shtools = np.array(
            [
                sh_coeffs.expand(colat=acc[1] / deg, lon=acc[2] / deg, a=acc[0])
                for acc in sph_coords
            ]
        )
        print(
            "Acceleration error wrt SHTOOLS",
            np.linalg.norm(acc_sim - acc_shtools, axis=0),
        )
    print("Took {:.2f} s".format(time() - ts))
else:
    # I the maximum degrees are the same, the computation is done below
    cnm_mat_est = cnm_mat_sim.copy()

Computing synthetic measurements...


100%|█████████████████████████████████████████| 121/121 [01:46<00:00,  1.13it/s]

Took 117.83 s


### Computing partials
The partial computation and the filtering are performed in the same way as in the previous notebook

In [7]:
print("Computing Normal equations...")
ts = time()

# measurements standard deviation. Can be a vector of size n_msr or a float.
msr_noise = 1e-4 * gm / np.power(r_0, 1 if use_potential else 2)
# in the scripts, the partials are computed up to a factor of gm
partials_scale = gm

# Computing the terms of the normal equations
# If sim_msr_vec is not None, it will be copied to msr_vec (and used to compute y),
# otherwise msr_vec and y are computed from the Stokes coefficients
N_mat, y, msr_vec = scripts.compute_normal_equations(
    cnm_mat_est, # coefficients used to simulate measurements (if sim_msr_vec is None)
    sph_coords, # measurements locations
    partials_func, # function for the partials computation
    r_0=r_0,
    batch_size=1000, # With 1000, about 2GB RAM for n_max_est=100 and 5e4 msr
    msr_noise=msr_noise,
    partials_scale=partials_scale,
    raw_msr_vec=sim_msr_vec, # synthetic measurements (if None they will be computed from cnm_mat_est)
    perturb_msr=perturb_msr,  # contaminate synthetic measurements with Gaussian noise
    rng=rng,
)
print("Took {:.2f} s".format(time() - ts))

Computing Normal equations...


100%|███████████████████████████████████████████| 88/88 [06:27<00:00,  4.40s/it]

Took 387.42 s


### Least-squares solution

In [8]:
cnm_vec_sim = cnm_mat_sim[*cnm_idx_sim.T]  # ground-truth vector of Stokes coefficients

# solution and covariance for the estimated parameters
# if compute_covariance is False, cov will be None 
cnm_vec_est, cov = scripts.solve_normal_equations(
    N_mat, y, compute_covariance=compute_covariance
)

if compute_covariance:
    cnm_vec_sigma = np.sqrt(np.diagonal(cov))  # formal errors
    corr_mat = cov / np.outer(cnm_vec_sigma, cnm_vec_sigma)  # correlations matrix

# difference between estimated and ground-truth Stokes coefficients
n_shared_coeffs = min(cnm_vec_sim.shape[0], cnm_vec_est.shape[0])
cnm_true_errors = cnm_vec_est[:n_shared_coeffs] - cnm_vec_sim[:n_shared_coeffs]


Solving via SVD...
Took 655.00 s


/tmp/ipykernel_84755/3525968641.py:10: RuntimeWarning: invalid value encountered in sqrt
  cnm_vec_sigma = np.sqrt(np.diagonal(cov))  # formal errors


### Plotting

In [9]:
val = msr_vec.reshape(-1, msr_dim)

# 3D trajectory with measurement points
fig_trj = scripts.plot_traj(state_mat, val, val_lbl=val_lbl)
fig_out_path = os.path.join(out_path, "sim_msr_plot_3D.html")
fig_trj.write_html(
            fig_out_path,
            include_mathjax="cdn",
            include_plotlyjs="cdn",
            config=dict({"scrollZoom": True}),
        )
IFrame(src=fig_out_path, width=1000, height=500)

In [10]:
# 2D plot of trajectory and measurements (in spherical coordinates)
plot_coords = sph_coords[:, [2, 1]]
plot_coords[:,1] = np.pi/2-plot_coords[:,1]
fig_grav_2d = scripts.plot_val_2d(
   plot_coords/deg, val, val_lbl=val_lbl
)
fig_out_path = os.path.join(out_path, "sim_msr_plot_2D.html")
fig_grav_2d.write_html(
            fig_out_path,
            include_mathjax="cdn",
            include_plotlyjs="cdn",
            config=dict({"scrollZoom": True}),
        )
IFrame(src=fig_out_path, width=800, height=500)

In [11]:
# Spectra of Sokes coefficients and their errors
cnm_spectrum_sim = np.array(
    [
        (n, np.linalg.norm(cnm_vec_sim[cnm_idx_sim[:, 1] == n]) / np.sqrt(2 * n + 1))
        for n in range(n_max_sim + 1)
    ]
)
cnm_spectrum_est = np.array(
    [
        (n, np.linalg.norm(cnm_vec_est[cnm_idx_est[:, 1] == n]) / np.sqrt(2 * n + 1))
        for n in range(n_max_est + 1)
    ]
)

# Line plots of the Cnm spectra
fig_spectrum = go.Figure()
fig_spectrum.add_traces(
    go.Scatter(
        x=cnm_spectrum_sim[2:, 0],
        y=cnm_spectrum_sim[2:, 1],
        mode="lines",
        line=dict(color="darkgrey", width=2),
        showlegend=True,
        name="Ground-truth",
    )
)
fig_spectrum.add_traces(
    go.Scatter(
        x=cnm_spectrum_est[2:, 0],
        y=cnm_spectrum_est[2:, 1],
        mode="lines",
        line=dict(color="royalblue", width=2),
        showlegend=True,
        name="Estimated",
    )
)
if compute_covariance:
    cnm_spectrum_sigma = np.array(
        [
            (
                n,
                np.linalg.norm(cnm_vec_sigma[cnm_idx_est[:, 1] == n])
                / np.sqrt(2 * n + 1),
            )
            for n in range(n_max_est + 1)
        ]
    )
    fig_spectrum.add_traces(
        go.Scatter(
            x=cnm_spectrum_sigma[2:, 0],
            y=cnm_spectrum_sigma[2:, 1],
            mode="lines",
            line=dict(color="royalblue", dash="dash", width=2),
            showlegend=True,
            name="Error",
        )
    )
fig_spectrum.update_layout(
    xaxis_title=r"$l_{\max}$", yaxis_title="Power spectrum", yaxis_type="log"
)
fig_out_path = os.path.join(out_path, "spectrum.html")
fig_spectrum.write_html(
            fig_out_path,
            include_mathjax="cdn",
            include_plotlyjs="cdn",
            config=dict({"scrollZoom": True}),
        )
IFrame(src=fig_out_path, width=800, height=500)

#### Error statistics
Here we evaluate the synthetic and the estimated Stokes coefficients over a same GLQ grid, computed for degree `n_max` (maximum of `n_max_sim` and `n_max_est`) and at a radial distance of $1.1R_0$

In [ ]:
print("Computing errors on GLQ grid...")
ts = time()

# constructing matrix from vector of estimated coefficients
cnm_mat_est = cnm_mat_est.astype(np.double)
cnm_mat_est[*cnm_idx_est.T] = cnm_vec_est

# cartesian (state_grid) and spherical(rtp_grid) coordinates of the GLQ grid points
state_grid, rtp_grid = scripts.points_from_grid(n_max, r=(r_0 * 1.1), use_GLQ_grid=True)

# forward gravity computation for ground-truth coefficients
# The function outputs are up to a factor of gm, so we multiply them by gm
pot_sim_grid, acc_sim_grid = [
    el * gm for el in scripts.compute_pot_acc(cnm_mat_sim, r_0, rtp_grid)
]
# forward gravity computation for estimated coefficients
pot_est_grid, acc_est_grid = [
    el * gm for el in scripts.compute_pot_acc(cnm_mat_est, r_0, rtp_grid)
]
print("Took {:.2f} s".format(time() - ts))

Computing errors on GLQ grid...


 59%|████████████████████████▋                 | 71/121 [00:19<00:23,  2.09it/s]

In [ ]:
# difference
err_grid = pot_sim_grid - pot_est_grid if use_potential else acc_sim_grid - acc_est_grid
err_grid = err_grid.reshape(-1, msr_dim)

rel_err = err_grid / ((pot_sim_grid if use_potential else acc_sim_grid) + 1e-20)

# Mean, median, RMS, and SNR of the errors
err_stats = [
    np.mean(err_grid),
    np.median(err_grid),
    np.linalg.norm(err_grid) / np.sqrt(err_grid.size),
]
err_stats.append(np.log10(np.abs(np.mean(1 / (rel_err + 1e-20)))))
print("Error stats: ")
print("Mean: {:.5g} - Median: {:.5g} -  RMS: {:.5g} - SNR (dB):{:.5g}".format(*err_stats))

# 3D plot of the errors
fig_diff =scripts.plot_traj(state_grid, err_grid, val_lbl="Error " + val_lbl)
fig_out_path = os.path.join(out_path, "true_errors_grid.html")
fig_diff.write_html(
        fig_out_path,
        include_mathjax="cdn",
        include_plotlyjs="cdn",
        config=dict({"scrollZoom": True}),
    )
IFrame(src=fig_out_path, width=1000, height=500)

### Correlation plot

In [ ]:
if compute_covariance and n_max<=10:
    # Heatmap of the correlation matrix
    _customdata = np.array(
        [
            "{}_{:d},{:d} - {}_{:d},{:d}".format(
                "C" if el_1[0] == 0 else "S",
                el_1[1],
                el_1[2],
                "C" if el_2[0] == 0 else "S",
                el_2[1],
                el_2[2],
            )
            for el_1 in cnm_idx_est
            for el_2 in cnm_idx_est
        ]
    ).reshape(corr_mat.shape)
    _hovertemplate="<b>%{customdata}</b><br>%{z:.3f}"
    fig_corr = go.Figure(
        data=go.Heatmap(
            z=corr_mat,
            customdata=_customdata,
            hovertemplate=_hovertemplate,
            colorscale="RdBu_r",
        )
    )
    fig_out_path = os.path.join(out_path, "correlations.html")
    fig_corr.write_html(
            fig_out_path,
            include_mathjax="cdn",
            include_plotlyjs="cdn",
            config=dict({"scrollZoom": True}),
        )
    IFrame(src=fig_out_path, width=500, height=500)

### Storing results

In [ ]:
# Saving Stokes coefficients in the SHADR format. Can be read via scripts.read_shadr or in SHTOOLS via sh.SHGravCoeffs.from_file
cnm_gt_file = "cnm_ground_truth"
scripts.write_SHADR(
    os.path.join(out_path, cnm_gt_file + ".txt"),
    cnm_vec_sim,
    cnm_map_sim,
    gm=gm,
    r_0=r_0,
)

cnm_sol_file = "cnm_estimated"
scripts.write_SHADR(
    os.path.join(out_path, cnm_sol_file + ".txt"),
    cnm_vec_est,
    cnm_map_est,
    gm=gm,
    r_0=r_0,
)

# Saving measurements evaluated on a spherical grid
grid_gt_file = "grid_ground_truth"
scripts.save_msr_grid(
    os.path.join(out_path, grid_gt_file + ".pkl"),
    rtp_grid,
    pot_sim_grid if use_potential else acc_sim_grid,
)
grid_est_file = "grid_estimated"
scripts.save_msr_grid(
    os.path.join(out_path, grid_est_file + ".pkl"),
    rtp_grid,
    pot_est_grid if use_potential else acc_est_grid,
)